In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os

# План.

    "Дом, который построил Джек".
    
### Задача 4
Задан текст (например, «Вот дом, который построил Джек»). Преложить алгоритм, который бы по нескольким предыдущим словам прогнозировал бы следующее слово. Проанализировать ошибку прогноза.


Обучим рекуррентную сеть на конституции России (там много про дома и про того, кто что и как должен строить). Будем генерировать слова посимвольно, потому что так проще.

# Чтение корпуса

In [2]:
#тут будет текст
corpora = ""

for fname in os.listdir("codex"):    
    with open("codex/"+fname) as fin:
        text = fin.read().decode('cp1251')
        corpora += text

In [3]:
print corpora[:1000]

---------------------------------------------


 Гражданский Кодекс
 Российской Федерации.
 Часть вторая



26 января 1996 года



 №14-ФЗ 

  Принят Государственной Думой 
  22 декабря 1995 года 


(в ред. Федеральных законов от 12.08.1996 №110-ФЗ, от 24.10.1997 №133-ФЗ, от 17.12.1999 №213-ФЗ, от 26.11.2002 №152-ФЗ, от 10.01.2003 №8-ФЗ, от 10.01.2003 №15-ФЗ, от 26.03.2003 №37-ФЗ, от 11.11.2003 №138-ФЗ, от 23.12.2003 №182-ФЗ, от 29.12.2004 №189-ФЗ, от 30.12.2004 №219-ФЗ, от 21.03.2005 №22-ФЗ, от 09.05.2005 №45-ФЗ, от 18.07.2005 №89-ФЗ, с изм., внесенными Федеральным законом от 26.01.1996 №15-ФЗ, Постановлением Конституционного Суда РФ от 23.12.1997 №21-П)





 Раздел IV. Отдельные виды обязательств



 Глава 30.
 Купля-продажа


 § 1. Общие положения о купле-продаже


 Статья 454.
 Договор купли-продажи

 1. По договору купли-продажи одна сторона (продавец) обязуется передать вещь (товар) в собственность другой стороне (покупателю), а покупатель 


In [4]:
tokens = set(corpora) #все уникальные буквы 

tokens = list(tokens)

In [6]:
#token_to_id = словарь символ-> его номер 
token_to_id = dict((t, n) for n, t in enumerate(tokens))

#id_to_token = словарь номер символа -> сам символ
id_to_token = dict((n,t) for t, n in token_to_id.iteritems())

#Преобразуем всё в токены
corpora_ids = np.array(map(lambda t: token_to_id[t], corpora))


In [7]:
def sample_random_batches(source, n_batches=10, seq_len=20):
    X_batch, y_batch = np.zeros((n_batches, seq_len)), np.zeros(n_batches)
    
    for i in xrange(n_batches):
        pos = np.random.randint(0, source.size - seq_len)
        X_batch[i, :] = source[pos:pos+seq_len]
        y_batch[i] = source[pos+seq_len]

    return X_batch, y_batch

In [8]:
sample_random_batches(corpora_ids, 4, 5)

(array([[ 39.,  69.,  44.,  62.,  57.],
        [ 44.,  60.,  44.,  39.,  15.],
        [ 63.,  73.,  60.,  12.,  44.],
        [ 57.,  43.,  15.,  89.,  89.]]), array([ 86.,  43.,  90.,  73.]))

# Константы

In [9]:
#длина последоватеьности при обучении (как далеко распространяются градиенты)
seq_length = 50

# Максимальный модуль градиента
grad_clip = 10

# Входные переменные

In [10]:
input_sequence = T.matrix('input sequence','int32')
target_values = T.ivector('target y')

# Соберём нейросеть

Вам нужно создать нейросеть, которая принимает на вход последовательность из seq_length токенов, обрабатывает их и выдаёт вероятности для seq_len+1-ого токена.

Общий шаблон архитектуры такой сети -


* Вход
* Обработка входа
* Рекуррентная нейросеть
* Вырезание последнего состояния
* Обычная нейросеть
* Выходной слой, который предсказывает вероятности весов.





Для обработки входных данных можно использовать либо EmbeddingLayer (см. прошлый семинар)

Как альтернатива - можно просто использовать One-hot энкодер

In [11]:
print (tokens)

[u'-', u'V', u'\u0410', u'\u0414', u'\u2116', u'\u0418', u'\u201c', u'\u0420', u'\u0424', u'\xa7', u'(', u'\xab', u',', u'0', u'4', u'\u0435', u'\xbb', u'\u043c', u'\u0440', u'7', u'3', u'\u0448', u'\u044c', u'9', u'2', u';', u'\u0413', u'\u0417', u'\u041b', u'\u041f', u'\xa0', u'\u0423', u'\u0427', u'+', u'\u042f', u'\u041c', u'\u0449', u'\u0433', u'\u0437', u'\u043b', u'\u201e', u'\u043f', u'\u0443', u'\u0447', u' ', u'\u044b', u'\u044f', u'"', u'c', u'\u0426', u'6', u'\n', u'\u0412', u'\u0416', u'\u041a', u'\u041e', u'\u0422', u'\u0430', u'\xa9', u'.', u'\u0432', u'\u0436', u'\u043a', u'\u0434', u'\u0442', u'\u0446', u'\u044a', u'\u044e', u'8', u'\u0438', u':', u'/', u'\u2013', u'\u043e', u'@', u'\r', u'\u0411', u'\u042e', u'\u0415', u'\u0419', u'\u041d', u'\u0444', u'\u0421', u'\u0425', u')', u'\u042d', u'\u0431', u'5', u'\u0439', u'\u043d', u'\u0441', u'!', u'\u0445', u'I', u'\u044d', u'%', u'N', u"'", u'P', u'\u0429', u'1', u'\u042b']


In [11]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

sky_net = lasagne.layers.EmbeddingLayer(l_in, input_size = len(tokens) + 1, output_size = 100)

rnn_sky_net = lasagne.layers.RecurrentLayer(sky_net, 50, only_return_final = True, grad_clipping = grad_clip)

sky_net = lasagne.layers.DenseLayer(rnn_sky_net, 300)

l_out = lasagne.layers.DenseLayer(sky_net, len(tokens), nonlinearity=lasagne.nonlinearities.softmax)

In [12]:
# Веса модели
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print weights

[W, input_to_hidden.W, input_to_hidden.b, hidden_to_hidden.W, W, b, W, b]


In [13]:
#network_output = Выход нейросети
network_output = lasagne.layers.get_output(l_out)

In [14]:
loss = lasagne.objectives.categorical_crossentropy(network_output, target_values).min()
updates = lasagne.updates.adam(loss, weights)

In [15]:
#обучение
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#функция потерь без обучения
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)

# Вероятности с выхода сети
probs = theano.function([input_sequence],network_output,allow_input_downcast=True)

# Генерирую!

* Для этого последовательно применяем нейронку к своему же выводу.

* Генерировать можно по разному -
 * случайно пропорционально вероятности,
 * только слова максимальной вероятностью
 * случайно, пропорционально softmax(probas*alpha), где alpha - "жадность"

In [16]:
def max_sample_fun(probs):
    return np.argmax(probs) 

In [17]:
import bisect

def proportional_sample_fun(probs):
    """Сгенерировать следующий токен (int32) по предсказанным вероятностям.
    
    probs - массив вероятностей для каждого токена
    
    Нужно вернуть одно целове число - выбранный токен - пропорционально вероятностям
    """
    cum = np.cumsum(probs)
    #return номер выбранного слова
    return bisect.bisect_left(cum, np.random.random())

In [18]:
# The next function generates text given a phrase of length at least SEQ_LENGTH.
# The phrase is set using the variable generation_phrase.
# The optional input "N" is used to set the number of characters of text to predict. 

def generate_sample(sample_fun,seed_phrase=None,N=200):
    '''
    Сгенерировать случайный текст при помощи сети

    sample_fun - функция, которая выбирает следующий сгенерированный токен
    
    seed_phrase - фраза, которую сеть должна продолжить. Если None - фраза выбирается случайно из corpora
    
    N - размер сгенерированного текста.
    
    '''

    if seed_phrase is None:
        start = np.random.randint(0,len(corpora)-seq_length)
        seed_phrase = corpora[start:start+seq_length]
        print "Using random seed:",seed_phrase
    while len(seed_phrase) < seq_length:
        seed_phrase = " "+seed_phrase
    if len(seed_phrase) > seq_length:
        seed_phrase = seed_phrase[len(seed_phrase)-seq_length:]
    assert type(seed_phrase) is unicode
           
    sample_ix = []
    x = map(lambda c: token_to_id.get(c,0), seed_phrase)
    x = np.array([x])

    for i in range(N):
        # Pick the character that got assigned the highest probability
        ix = sample_fun(probs(x).ravel())
        # Alternatively, to sample from the distribution instead:
        # ix = np.random.choice(np.arange(vocab_size), p=probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1] = x[:,1:]
        x[:,seq_length-1] = 0
        x[0,seq_length-1] = ix 

    random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
    print("----\n %s \n----" % random_snippet)

# Обучение модели

В котором вы можете подёргать параметры или вставить свою генерирующую функцию.



In [19]:
print("Training ...")

#сколько всего эпох
n_epochs=100

# раз в сколько эпох печатать примеры 
batches_per_epoch = 1000

#сколько цепочек обрабатывать за 1 вызов функции обучения
batch_size=100

for epoch in xrange(n_epochs):
    print "Генерируем текст в пропорциональном режиме"
    generate_sample(proportional_sample_fun,None)
    
    print "Генерируем текст в жадном режиме (наиболее вероятные буквы)"
    generate_sample(max_sample_fun,None)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_random_batches(corpora_ids,batch_size,seq_length)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))

Training ...
Генерируем текст в пропорциональном режиме
Using random seed: итражный суд не принимает отказ истца от иска, уме
----
 итражный суд не принимает отказ истца от иска, уме;г1ФпфсЧя8ш/VЯы4КЭУъщщ«:“лу
Фд©ЦОСХА%З-Мн№з"6нтА0фОХжМдVХшз«яо№ 7IйэМNЖБPйАЗэP7ф:3р@««ъР«c7МРЭТыЧш.РДхб»«п 1)V'ЕУЩ3„вауоP8'P5Мж,'ЕзЩпЩ@жIГ)  (§ГыРМ№я“§Г
Чш–)9.ОЧНПЮ©ЦЙ'лд)9гЫПсан+PюЧюДЕ
оЧ2м
+оХЗcщ6лЯ 
----
Генерируем текст в жадном режиме (наиболее вероятные буквы)
Using random seed: ишение права управления транспортными средствами н
----
 ишение права управления транспортными средствами нСуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоPПуоP 
----
Epoch 0 average loss = 0.118729468153
Генерируем текст в пропорциональном режиме
Using random seed:  отношении которого составлен протокол, а также оп
----
  отношении которого составлен протокол, а также опооооооооооо

KeyboardInterrupt: 

# Анализ ошибки показает, что в начале сети удобнее генерировать популярные буквы, так в среднем кросс-энтропия меньше. Иногда она угадывает, и кросс-энтропия получается ноль. Иногда нет.

# Конституция нового мирового правительства

In [20]:
seed = u"Каждый человек должен"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
                              Каждый человек долженооооо+о6оооаооооооооощоооо"/ооооооХ;оМ;оооосоооао оБаооооооооаооо6то%)ооозоооо оороооооооо«4оооооо ооооо(оооооооСоорооцоооояооооооооойоооооожоооооо о" ооо,ооооооооcЕоВэоооо оооооооооооооЙ ооохоооооооооо оРФ ооооороооооодоооо« оооооооооооо ооооХаооаоФ»/ ооооооао оооооооооооооооооооооохоооо ооооЙооооо 
----


In [21]:
seed = u"В случае неповиновения"
sampling_fun = proportional_sample_fun
result_length = 300

generate_sample(sampling_fun,seed,result_length)

----
                             В случае неповиновенияоооооооооо,ооооооооо ооооооЕоооо№@3оооооооооорооооооцоохоооооЮ;о1дооооо оооо о оойооооо ооаооо©Т»ьоооооо“2ооо6о–офоооооожооооооо5 ооP№ЕоЭоооРNаооооооо оокоооото Рооооооо ооЛЮОооооооо оооЧСоооо ооооо„cоооооо оооооооо о оооХЦ'ооооооооооооПооооооао оооооБоооооооодооо©оооооооооооооооо9ЕогоооооооПоодо оо 
----
